In [72]:
# install libraries
# !pip install hazm
# !pip install pandas
# !pip install gensim
# !pip install tqdm
# !pip3 install sklearn
# !pip3 install seaborn

In [33]:
# libraries
import codecs
import os
import json
import tqdm
import random
import pickle
from collections import Counter
from nltk import FreqDist
import pandas as pd
import numpy as np
import itertools
from hazm import *
import matplotlib.pyplot as plt
from gensim.models.fasttext import FastText
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

In [34]:
import json

DIRNAME = '../new-dataset-with-category'
data = []
for i in range(1, 31):
    with open(f'{DIRNAME}/namnak-{i}.json', 'r', encoding="utf-8") as f:
        data.extend(json.loads(f.read()))


In [35]:
len(data)

7053

In [36]:
category_dictionary = {
    'سلامت روان' : 0,
    'دهان و دندان' : 1,
    'پوست و مو' : 2,
    'تغذیه' : 3,
    'سلامت خانواده' : 4,
    'سلامت جنسی' : 5,
    'پیشگیری و بیماریها' : 6
}

doc_num = {_: 0 for _ in category_dictionary.values()}
doc_num_threshold = 350

selected_data = []

for i in tqdm.tqdm(range(0, len(data))):
    doc = data[i]
    if doc['category'] in category_dictionary.keys() and doc_num[category_dictionary[doc['category']]] < doc_num_threshold:
        doc_num[category_dictionary[doc['category']]] += 1
        selected_data.append(doc)
data = selected_data

100%|██████████████████████████████████████████| 7053/7053 [00:00<00:00, 1149278.40it/s]


In [37]:
len(data)

2434

### normalization

In [38]:
# normalization
from hazm import *

normalizer = Normalizer()

normalized_data = []
for item in tqdm.tqdm(data):
    normalized_data.append({"title" : normalizer.normalize(item['title']), "text":normalizer.normalize(item['abstract'] + " " + " ".join(item['paragraphs'])), "link": item['link']})

100%|██████████████████████████████████████████████| 2434/2434 [00:02<00:00, 976.03it/s]


In [39]:
import random
print(random.sample(normalized_data, 10)[0]['text'][:1000])

ارتودنسی روشی برای زیبایی و مرتب کردن دندان‌ها می‌باشد اما این روش عوارض جانبی در داخل یا بیرون دهان ایجاد می‌کند که باید قبل از ارتودنسی بدانید. انواع عوارض ارتودنسی را بشناسید درمان ارتودنسی به اصلاح نامرتبی‌های دندان از طریق براکت‌های ثابت و یا متحرک می‌پردازد. جدا از اثر زیبایی می‌توان گفت که این درمان امکان آسیب رسانی و عوارض جانبی را نیز دارد که کاملا ناخواسته است. عوارض جانبی آن می‌تواند در داخل دهان یا بیرون دهان و یا کل بدن اتفاق بیافتد. اثرات جانبی در داخل دهان شامل موارد زیر است: اثرات روی دندان اثر روی بافت لثه اثرات روی بافت نرم اثرات روی دندان کلسیم و یا مواد آهکی روی سطح دندان باعث ایجاد لکه‌های سفید می‌شود و یکی از مشکلات رایج داخل دهانی حین ارتودنسی می‌باشد. این مواد آهکی روی سطح دندان به دلیل اسید تولید شده از پلاکت‌های دارای باکتری ایجاد می‌شود. پروسه تشکیل مواد آهکی روی دندان بسیار سریع اتفاق می‌افتد و اغلب یک ماه بعد از درمان ارتودنسی مخصوصا ارتودنسی‌های ثابت پدیدار می‌گردد. بنابراین بسیار مهم است که بهداشت دهانی را به طور عالی در پیش بگیریم. همچنین از مصرف مواد اس

### stopwords

In [40]:
# Persian Stopwords
# https://github.com/sobhe/hazm/blob/master/hazm/data/stopwords.dat
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

In [41]:
def create_dataframe_freq(data, stopwords=[]):
    tokens = []
    for item in tqdm.tqdm(data):
        tokens.extend([_ for _ in word_tokenize(item['text'])  if _ not in stopwords])
    return pd.DataFrame(FreqDist(tokens).most_common(30)), tokens

In [42]:
freq_analysis, tokens = create_dataframe_freq(normalized_data, stopwords)

100%|██████████████████████████████████████████████| 2434/2434 [00:04<00:00, 543.38it/s]


In [43]:
freq_analysis

,0,1
0,.,92860
1,،,66676
2,:,12152
3,استفاده,10567
4,پوست,8431
5,بدن,7243
6,درمان,6926
7,مصرف,6892
8,دندان,6692
9,آب,6175


In [44]:
custom_stop_words = [normalizer.normalize(x.strip()) for x in codecs.open('custom_stopwords.txt','r','utf-8').readlines()]

In [45]:
total_stop_words = custom_stop_words + stopwords

In [46]:
freq_analysis, tokens = create_dataframe_freq(normalized_data, total_stop_words)

100%|██████████████████████████████████████████████| 2434/2434 [00:04<00:00, 563.92it/s]


In [47]:
freq_analysis

,0,1
0,استفاده,10567
1,پوست,8431
2,بدن,7243
3,درمان,6926
4,مصرف,6892
5,دندان,6692
6,آب,6175
7,کاهش,6039
8,کمک,5728
9,مواد,5575


In [48]:
all_doc_tokens = []
for item in tqdm.tqdm(normalized_data):
    all_doc_tokens.append([_ for _ in word_tokenize(item['text'])  if _ not in total_stop_words])

100%|██████████████████████████████████████████████| 2434/2434 [00:04<00:00, 565.27it/s]


## Lematize

In [49]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt ==1:
        return stemmer.stem(tok)
    if opt ==2:
        return lemmatizer.lemmatize(tok)

In [50]:
opt = 2
tokens_after_lem = []

for tokens in all_doc_tokens:
    tokens_after_lem.append([get_lemma_set(token, opt) for token in tokens])

## tfidf

In [51]:
tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words=total_stop_words, ngram_range=(1,1))
doc_term_mat = tfidfvectorizer.fit_transform([' '.join(doc) for doc in tokens_after_lem])

/opt/homebrew/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آید', 'توان', 'تواند', 'توانند', 'رسد', 'رود', 'سال', 'نمی', 'های', 'گوید', 'گویند'] not in stop_words.
  warnings.warn(


In [52]:
# save tfidf vectorizer
with open('../models/tfidfvectorizer-clustering.pk', 'wb') as fin:
    pickle.dump(tfidfvectorizer, fin)

In [53]:
# load tfidf vectorizer
with open('../models/tfidfvectorizer-clustering.pk', 'rb') as fin:
    tfidfvectorizer_loaded = pickle.load(fin)

In [54]:
tfidfvectorizer_loaded.transform(['ریزش مو در مردان'])

<1x25290 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [55]:
doc_term_mat.shape

(2434, 25290)

In [56]:
# save matrix
os.makedirs('../models', exist_ok=True)
sparse.save_npz("../models/doc-term-tfidf-clustering.npz", doc_term_mat)

In [57]:
# load matrix
doc_term_mat = sparse.load_npz("../models/doc-term-tfidf-clustering.npz")

In [58]:
doc_term_mat.shape

(2434, 25290)

In [59]:
tfidfvectorizer.idf_[1]

8.104554854956746

In [60]:
tfidfvectorizer.get_feature_names_out()[1]

'_آسم'

In [61]:
np.where(tfidfvectorizer.get_feature_names_out() == 'آب')[0][0]

1632

In [62]:
doc_term_mat[4].toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [63]:
import codecs
import json
import numpy as np
from hazm import *
from gensim.models.fasttext import FastText


class TfidfEmbedingModel:
    def __init__(self):
        self.doc_term_mat = sparse.load_npz("../models/doc-term-tfidf-clustering.npz")
        with open('../models/tfidfvectorizer-clustering.pk', 'rb') as fin:
            self.tfidfvectorizer = pickle.load(fin)
        self.normalizer = Normalizer()
        stopwords = [self.normalizer.normalize(x.strip()) for x in codecs.open('../stopwords/stopwords.txt','r','utf-8').readlines()]
        custom_stop_words = [self.normalizer.normalize(x.strip()) for x in codecs.open('../stopwords/custom_stopwords.txt','r','utf-8').readlines()]
        self.total_stop_words = custom_stop_words + stopwords
        
    def get_doc_embeding(self, doc, index):
        return ((self.get_text_embeding(index) * 19) + self.get_title_embeding(doc['title'])) / 20
    
    def get_text_embeding(self, index):
        return self.doc_term_mat[index].toarray()[0]
            
    def get_title_embeding(self, title):
        tokens = [_ for _ in word_tokenize(self.normalizer.normalize(title)) if _ not in self.total_stop_words]
        emb = self.tfidfvectorizer.transform([' '.join(tokens)])
        return emb[0].toarray()[0]

In [64]:
tfidf_embeding_model = TfidfEmbedingModel()

In [65]:
category_dictionary = {
    'سلامت روان' : 0,
    'دهان و دندان' : 1,
    'پوست و مو' : 2,
    'تغذیه' : 3,
    'سلامت خانواده' : 4,
    'سلامت جنسی' : 5,
    'پیشگیری و بیماریها' : 6
}

doc_num = {_: 0 for _ in category_dictionary.values()}
doc_num_threshold = 350

X_train = list()
Y_test = list()

for i in tqdm.tqdm(range(0, len(data))):
    doc = data[i]
    if doc['category'] in category_dictionary.keys() and doc_num[category_dictionary[doc['category']]] < doc_num_threshold:
        doc_num[category_dictionary[doc['category']]] += 1
        Y_test.append(category_dictionary[doc['category']])
        X_train.append(tfidf_embeding_model.get_doc_embeding(doc, i))

100%|█████████████████████████████████████████████| 2434/2434 [00:00<00:00, 2564.31it/s]


In [66]:
kmeans = KMeans(n_clusters=7, n_init=50, max_iter=1000, tol=1e-8).fit(X_train)
Y_predicted = kmeans.predict(X_train)
Y_center = [kmeans.cluster_centers_[y] for y in Y_predicted]
for i in range(0,7):
    print(f'cluster{i} : {Y_predicted.tolist().count(i)}')

cluster0 : 483
cluster1 : 121
cluster2 : 283
cluster3 : 517
cluster4 : 212
cluster5 : 89
cluster6 : 729


In [67]:
print(f'RSS = {kmeans.inertia_}')

RSS = 1981.7385132457648


In [68]:
def purity_score(y_true, y_pred):
    contin_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(contin_matrix, axis=0)) / np.sum(contin_matrix) 

In [69]:
purity_score(Y_test, Y_predicted)

0.6261298274445357

In [70]:
metrics.silhouette_score(X_train, Y_predicted)

0.03852207731586154

In [71]:
davies_bouldin_score(X_train, Y_predicted)

5.72715910762129